In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler(feature_range=(0, 1), copy=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error # 平均絶対誤差
#from sklearn.metrics import mean_squared_error # 平均二乗誤差
#from sklearn.metrics import mean_squared_log_error # 対数平均二乗誤差
from sklearn.metrics import r2_score # 決定係数
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import missingno as msno
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

# 1. Import data

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/test.csv")

In [ ]:
# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.info()

In [ ]:
train

In [ ]:
train.describe()

In [ ]:
test

In [ ]:
test.describe()

# 2. EDA

In [ ]:
# Colors to be used for plots
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen", "lightseagreen",
          "cornflowerblue", "mediumpurple", "palevioletred", "lightskyblue", "sandybrown",
          "yellowgreen", "indianred", "lightsteelblue", "mediumorchid", "deepskyblue"]

In [ ]:
# Add columns

train['date_time'] = pd.to_datetime(train['time'])
train['month'] = train['date_time'].dt.month
train['day'] = train['date_time'].dt.day
train['dayofweek'] = train['date_time'].dt.dayofweek
train['hour'] = train['date_time'].dt.hour
#train['minute'] = train['date_time'].dt.minute
train['minute_c'] = train['date_time'].dt.hour * 60 + train['date_time'].dt.minute

test['date_time'] = pd.to_datetime(test['time'])
test['month'] = test['date_time'].dt.month
test['day'] = test['date_time'].dt.day
test['dayofweek'] = test['date_time'].dt.dayofweek
test['hour'] = test['date_time'].dt.hour
#test['minute'] = test['date_time'].dt.minute
test['minute_c'] = test['date_time'].dt.hour * 60 + test['date_time'].dt.minute

In [ ]:
sns.catplot(x="month", y="congestion", data=train, kind="violin")
plt.show()
sns.catplot(x="day", y="congestion", data=train, kind="violin")
plt.show()
sns.catplot(x="dayofweek", y="congestion", data=train, kind="violin")
plt.show()
sns.catplot(x="hour", y="congestion", data=train, kind="violin")
plt.show()

In [ ]:
# Search for missing data

msno.matrix(df=train, figsize=(10,6), color=(0,.3,.3))

In [ ]:
msno.matrix(df=test, figsize=(10,6), color=(0,.3,.3))

# 3. Preprosessing

In [ ]:

# Week factors
train["is_monday"] = (train["date_time"].dt.dayofweek == 0).astype("int")
test["is_monday"] = (test["date_time"].dt.dayofweek == 0).astype("int")

# Time zone factors
ary =[]
for i in range(len(train["date_time"])):
    if (12 <= train["hour"][i] <= 23):#12～23
        ary.append(1)
    else:
        ary.append(0)
train["target_time_zone"] = ary

ary =[]
for i in range(len(test["date_time"])):
    if (12 <= test["hour"][i] <= 23):
        ary.append(1)
    else:
        ary.append(0)
test["target_time_zone"] = ary


In [ ]:
'''
 Converts the direction to a vector that represents the coordinates.

 NB => N:1,     E:0,     S:0,     W:0
 NE => N:0.876, E:0.876, S:0,     W:0
 EB => N:0,     E:1,     S:0,     W:0
 SE => N:0,     E:0.876, S:0.876, W:0
 SB => N:0,     E:0,     S:1,     W:0
 SW => N:0,     E:0,     S:0.876, W:0.876
 WB => N:0,     E:0,     S:0,     W:1
 NW => N:0.876, E:0,     S:0,     W:0.876
 
 (1 / √2 ≒ 0.876)
 
'''

ary_N = []
ary_E = []
ary_S = []
ary_W = []
for i in range(len(train["direction"])):
    if (train["direction"][i] == 'NB'):
        ary_N.append(1)
        ary_E.append(0)
        ary_S.append(0)
        ary_W.append(0)
    elif (train["direction"][i] == 'NE'):
        ary_N.append(0.876)
        ary_E.append(0.876)
        ary_S.append(0)
        ary_W.append(0)
    elif (train["direction"][i] == 'EB'):
        ary_N.append(0)
        ary_E.append(1)
        ary_S.append(0)
        ary_W.append(0)
    elif (train["direction"][i] == 'SE'):
        ary_N.append(0)
        ary_E.append(0.876)
        ary_S.append(0.876)
        ary_W.append(0)
    elif (train["direction"][i] == 'SB'):
        ary_N.append(0)
        ary_E.append(0)
        ary_S.append(1)
        ary_W.append(0)
    elif (train["direction"][i] == 'SW'):
        ary_N.append(0)
        ary_E.append(0)
        ary_S.append(0.876)
        ary_W.append(0.876)
    elif (train["direction"][i] == 'WB'):
        ary_N.append(0)
        ary_E.append(0)
        ary_S.append(0)
        ary_W.append(1)
    elif (train["direction"][i] == 'NW'):
        ary_N.append(0.876)
        ary_E.append(0)
        ary_S.append(0)
        ary_W.append(0.876)
train["N"] = ary_N
train["E"] = ary_E
train["S"] = ary_S
train["W"] = ary_W

ary_N = []
ary_E = []
ary_S = []
ary_W = []
for i in range(len(test["direction"])):
    if (test["direction"][i] == 'NB'):
        ary_N.append(1)
        ary_E.append(0)
        ary_S.append(0)
        ary_W.append(0)
    elif (test["direction"][i] == 'NE'):
        ary_N.append(0.876)
        ary_E.append(0.876)
        ary_S.append(0)
        ary_W.append(0)
    elif (test["direction"][i] == 'EB'):
        ary_N.append(0)
        ary_E.append(1)
        ary_S.append(0)
        ary_W.append(0)
    elif (test["direction"][i] == 'SE'):
        ary_N.append(0)
        ary_E.append(0.876)
        ary_S.append(0.876)
        ary_W.append(0)
    elif (test["direction"][i] == 'SB'):
        ary_N.append(0)
        ary_E.append(0)
        ary_S.append(1)
        ary_W.append(0)
    elif (test["direction"][i] == 'SW'):
        ary_N.append(0)
        ary_E.append(0)
        ary_S.append(0.876)
        ary_W.append(0.876)
    elif (test["direction"][i] == 'WB'):
        ary_N.append(0)
        ary_E.append(0)
        ary_S.append(0)
        ary_W.append(1)
    elif (test["direction"][i] == 'NW'):
        ary_N.append(0.876)
        ary_E.append(0)
        ary_S.append(0)
        ary_W.append(0.876)
test["N"] = ary_N
test["E"] = ary_E
test["S"] = ary_S
test["W"] = ary_W

In [ ]:
train

In [ ]:
test

In [ ]:
corr = train.corr().round(2)
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
corr = test.corr().round(2)
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Concat train and test
all = pd.concat([train,test],ignore_index=True)

In [ ]:
all.drop(columns=['row_id', 'time', 'direction', 'date_time']).describe().T\
        .style.bar(subset=['mean'], color=px.colors.qualitative.G10[0])\
        .background_gradient(subset=['std'], cmap='Greens')\
        .background_gradient(subset=['50%'], cmap='BuGn')

In [ ]:
'''
# Scaling
columns = ['x', 'y', 'month', 'day', 'dayofweek', 'hour', 'is_monday', 'target_time_zone', 'N', 'E', 'S', 'W']
all_scale = mmscaler.fit_transform(all.drop(columns=['row_id', 'time', 'direction', 'congestion', 'date_time']))
all_scale = pd.DataFrame(all_scale, columns=columns)
# Split all for train and test
train_scale = all_scale.iloc[train.index[0]:train.index[-1]+1]
test_scale = all_scale.iloc[train.index[-1]+1:]
train_scale['congestion'] = train['congestion']
'''


# 4. Modeling

In [ ]:
#X = train_scale.drop(columns=['congestion'])
X = train.drop(columns=['row_id', 'time', 'direction', 'date_time', 'congestion'])
#value = train_scale['congestion']
value = train['congestion']

In [ ]:
X_train, X_test, t_train, t_test = train_test_split(X, value, test_size=0.2, random_state=0)

lgb_train = lgb.Dataset(X_train, t_train)
lgb_eval = lgb.Dataset(X_test, t_test, reference=lgb_train)

params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',#rmse,l2
        'learning_rate': 0.1,#0.5
        'num_leaves': 747,#13:6.122255830446027,23:6.093490341192029,
#                         33:6.090168882097454,43:6.086209247739203,
#                         74:6.085693645383924,194:6.083869097762618,
#                         294:6.082327481257648,394:5.946850782805147,
#                         494:5.938235206137852,594:5.937642052983539,
#                         677:5.931394058308976,747:5.930571255674824,
#                         1247:5.929094465522741,2247:5.924352967082318,
#                         3247:5.917061218473121,4247:5.915041235290781,
#                         4277:5.913973683944743,
        'max_bin': 50,#50
        'num_iterations': 20000,
        'verbosity': -1
}

model = lgb.train(
    params,
    train_set=lgb_train,
    valid_sets=lgb_eval,
    early_stopping_rounds=100,
    verbose_eval=100
)


# Verification

pred = model.predict(X_test)

mae = mean_absolute_error(t_test, pred) # mae(平均絶対誤差)の算出
print('mean_absolute_error : {}'.format(mae))

#r2 = r2_score(t_test,pred)
#print('R2    : {}'.format(r2))

# 5. Prediction

In [ ]:
#X_test = test_scale
X_test = test.drop(columns=['row_id', 'time', 'direction', 'date_time'])
sample_submission['congestion'] = model.predict(X_test)
sample_submission

# 6. Make submission file

In [ ]:
sample_submission.to_csv('submission.csv', index=False)